In [82]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = Chroma("langchain_store", embeddings)

In [72]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
df_exp = df_exp.head()


In [83]:
for description in df_exp.description:
    print(description)
    vectorstore.add_texts([description])

Sales managers that resolve customer complaints regarding sales and service.
Sales managers that monitor customer preferences to determine focus of sales efforts.
Sales managers that determine price schedules and discount rates.
Sales managers that review operational records and reports to project sales and determine profitability.
Sales managers that confer or consult with department heads to plan advertising services and to secure information on equipment and customer specifications.


In [84]:
vectorstore.similarity_search('sales managers')

[Document(page_content='Sales managers that resolve customer complaints regarding sales and service.'),
 Document(page_content='Sales managers that resolve customer complaints regarding sales and service.'),
 Document(page_content='Sales managers that resolve customer complaints regarding sales and service.'),
 Document(page_content='Sales managers that monitor customer preferences to determine focus of sales efforts.')]

In [85]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=2000)
retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



prompt = ChatPromptTemplate.from_template(
    "these are example task: {context}.\n\n {question}."
)

print(prompt)

# Configure the Rag (Retrieval-Augmented Generation) Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



prompt = "what are tasks for sales managers"
print(rag_chain.invoke(prompt))




input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='these are example task: {context}.\n\n {question}.'))]
1. Reviewing operational records and reports.
2. Projecting sales.
3. Determining profitability.
4. Resolving customer complaints regarding sales and service.


In [ ]:
#does vector store consider examples from prompt as well